In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError
from lat_lon_parser import parse
from dotenv import load_dotenv
import os
import mysecrets
load_dotenv()  # Load .env into environment

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from lat_lon_parser import parse # for decimal coordinates 

def cities_dataframe(cities):

  city_data = []

  for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    headers = {'User-Agent': 'Chrome/134.0.0.0'}

    response = requests.get(url, headers=headers)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
    city_latitude = city_soup.find(class_="latitude").get_text()
    city_longitude = city_soup.find(class_="longitude").get_text()
    country = city_soup.find(class_="infobox-data").get_text()

   city_data.append({"city_name": city,
                    "country": country,
                    "latitude": parse(city_latitude), # latitude in decimal format
                    "longitude": parse(city_longitude) # longitude in decimal format
                    })

  return pd.DataFrame(city_data)


In [2]:
list_cities= ['berlin', 'Munich', 'Hamburg']

cities_df = cities_dataframe(list_cities)
cities_df

,city_name,country,latitude,longitude
0,berlin,Germany,52.5200,13.405
1,Munich,Germany,48.1375,11.575
2,Hamburg,Germany,53.5500,10.000


In [5]:
def connect_to_gans_local():
  schema = "gans_sample_solution"
  host = "127.0.0.1"
  user = "root"
  password = 'my_password'
  port = 3306
  
  connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
  return connection_string
connection_string = connect_to_gans_local()

In [6]:
cities_df.to_sql(name='cities', con=connection_string, if_exists='append', index=False)

3

In [7]:
def scrape_and_send_city_data(cities_list):

    connection_string = connect_to_gans_local()

   
    cities_data = []

    for city in cities_list:
        url = f"https://www.wikipedia.org/wiki/{city}"
        headers = {'User-Agent': 'Chrome/134.0.0.0'}

        response = requests.get(url, headers=headers)
        city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
        city_latitude = city_soup.find(class_="latitude").get_text()
        city_longitude = city_soup.find(class_="longitude").get_text()
        country = city_soup.find(class_="infobox-data").get_text()

    # keep track of data per city
        cities_data.append({"city_name": city,
                    "country": country,
                    "latitude": parse(city_latitude), # latitude in decimal format
                    "longitude": parse(city_longitude) # longitude in decimal format
                    })


    # Combine all rows into data frame
    cities_df = pd.DataFrame(cities_data)
# Send data to database
    cities_df.to_sql('cities',
                     if_exists='append',
                     con=connection_string,
                     index=False)

In [12]:
# Scrapes data for cities populations table
from datetime import datetime # to get today's date
def scrape_and_send_population_data():

    # Get connection string
    connection_string = connect_to_gans_local()

    cities_table = pd.read_sql('cities',
                               con=connection_string)

    populations_data = []

    for city in cities_table['city_name']:

        # Create url
        url = f"https://en.wikipedia.org/wiki/{city}"
        header= {'User-Agent':"Chrome 134.0.0.0"}
        # Pull HTML
        response = requests.get(url, headers=header)
        city_soup = BeautifulSoup(response.content, 'html.parser')

        # Locate and extract data
        pop = city_soup.find(string="Population").find_next('td').get_text()
        pop_digits = re.sub(r'\D', '', pop)  # removes everything except 0–9
        pop_int = int(pop_digits)
        today = datetime.today().strftime("%d.%m.%Y")
        today = pd.to_datetime(today, format="%d.%m.%Y")

        
        population_data = {
            'city_name': city,
            'population': pop_int,
            'time_stamp': today
        }
        # Add row to list
        populations_data.append(population_data)

   
    populations_df = pd.DataFrame(populations_data)

   
    city_populations_df = (
        populations_df
        .merge(cities_table,
               on='city_name',
               how='inner')
        [['city_id', 'population', 'time_stamp']]
    )
    # Send data to database
    city_populations_df.to_sql('populations',
                               if_exists='append',
                               con=connection_string,
                               index=False)

In [13]:
cities_list = ['Berlin', 'Munich', 'Hamburg']
# Scrapes and sends data for list of cities
scrape_and_send_city_data(cities_list)
# Scrapes and sends population data for all cities in database
scrape_and_send_population_data()

In [24]:
import pandas as pd
import requests
from pytz import timezone
from datetime import datetime

def retrieve_and_send_data():
  connection_string = create_connection_string()
  cities_df = fetch_cities_data(connection_string)
  weather_df = fetch_weather_data(cities_df)
  store_weather_data(weather_df, connection_string)
  return "Data has been updated"

def create_connection_string():
  schema = "gans_sample_solution"
  host = "127.0.0.1"
  user = "root"
  password = mysecrets.passwrod
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)
  cities_df = fetch_cities_data(connection_string)
    
def fetch_weather_data(cities_df):
  berlin_timezone = timezone('Europe/Berlin')
  API_key = 'Api_key'
  weather_items = []

  for _, city in cities_df.iterrows():
      latitude = city["latitude"]
      longitude = city["longitude"]
      city_id = city["city_id"]

      url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")
      response = requests.get(url)
      weather_data = response.json()

      retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

      for item in weather_data["list"]:
          weather_item = {
              "city_id": city_id,
              "forecast_time": item.get("dt_txt"),
              "temperature": item["main"].get("temp"),
              "forecast": item["weather"][0].get("main"),
              "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
              "wind_speed": item["wind"].get("speed"),
              "data_retrieved_at": retrieval_time
          }
          weather_items.append(weather_item)

  weather_df = pd.DataFrame(weather_items)
  weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
  weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

  weather_df = pd.DataFrame(weather_items)
  return weather_df

def store_weather_data(weather_df, connection_string):
  weather_df.to_sql('weather',
                    if_exists='append',
                    con=connection_string,
                    index=False)




In [25]:
retrieve_and_send_data()

'Data has been updated'